In [ ]:
import json 
import pandas as pd

In [ ]:
import re

In [ ]:
f = open('Data/scorecard.json',) 
data = json.load(f)   

In [ ]:
type(data)

In [ ]:
data_keys = list(data.keys())

In [ ]:
len(data.keys())

In [ ]:
players={}

In [ ]:
for i in data_keys:
    if(data[i]!=None):
            for j in data[i]['BATTING1']:
                players.setdefault(i,[]).append(j[0])
            for k in data[i]['BATTING2']:
                players.setdefault(i,[]).append(k[0])
            for l in data[i]['BOWLING1']:
                players.setdefault(i,[]).append(l[0])
            for m in data[i]['BOWLING2']:
                players.setdefault(i,[]).append(m[0])

In [ ]:
players['1046']

In [ ]:
match = pd.read_csv("Data/matches.csv")

In [ ]:
len(match)

In [ ]:
len(match.MatchCode.unique())

In [ ]:
see_patterns=[]

In [ ]:
for i in data_keys:
    if(data[i]!=None):
            for j in data[i]['BATTING1']:
                see_patterns.append(j[1])
            for k in data[i]['BATTING2']:
                see_patterns.append(j[1])

In [ ]:
see_patterns

### Getting Fielding Data (unmapped)

In [ ]:
team1={}   # Match id: [(player id, dismissal)]

In [ ]:
team2={}

In [ ]:
data_keys = list(data.keys())

In [ ]:
for i in data_keys:
        if(data[i]!=None):
            for j in data[i]['BATTING1']:
                team2.setdefault(i,[]).append((j[0],j[1].lower()))
            for k in data[i]['BATTING1']:
                team1.setdefault(i,[]).append((k[0],k[1].lower()))

In [ ]:
len(team1)

In [ ]:
len(team2)

In [ ]:
team1_keys=team1.keys()
team2_keys=team2.keys()

In [ ]:
fielders={}

In [ ]:
for i in team2_keys:
    
    for j in team2[i]:
        search_in=j[1].replace('\u2020','')
        pattern_catch = re.search('^c\s+',search_in)
        pattern_st = re.search('^st\s+',search_in)
        
        if pattern_catch:
            
            splitted = re.split('c\s+|\s+b\s+',search_in)
            catch_list = list(filter(None, splitted))
            if (len(catch_list)<2):
                catch_list.append(catch_list[0])
            if (catch_list[0]=='&'):
                catch_list[0]=catch_list[1]
            fielders.setdefault(i,[]).append((j[0],{'c2':catch_list[0],'b2':catch_list[1]}))
            
            pattern_catch=None
            
        elif pattern_st:
            splitted = re.split('st\s+|\s+b\s+',search_in)
            st_list = list(filter(None, splitted))
            if (len(st_list)<2):
                st_list.append(st_list[0])
            if (st_list[0]=='&'):
                st_list[0]=st_list[1]
            fielders.setdefault(i,[]).append((j[0],{'st2':st_list[0],'b2':st_list[1]}))
                
            pattern_st=None

In [ ]:
for i in team1_keys:
    
    for j in team1[i]:
        search_in=j[1].replace('\u2020','')
        pattern_catch = re.search('^c\s+',search_in)
        pattern_st = re.search('^st\s+',search_in)
        
        if pattern_catch:
            
            splitted = re.split('c\s+|\s+b\s+',search_in)
            catch_list = list(filter(None, splitted))
            if (len(catch_list)<2):
                catch_list.append(catch_list[0])
            if (catch_list[0]=='&'):
                catch_list[0]=catch_list[1]
            fielders.setdefault(i,[]).append((j[0],{'c1':catch_list[0],'b1':catch_list[1]}))
            
            pattern_catch=None
            
        elif pattern_st:
            splitted = re.split('st\s+|\s+b\s+',search_in)
            st_list = list(filter(None, splitted))
            if (len(st_list)<2):
                st_list.append(st_list[0])
            if (st_list[0]=='&'):
                st_list[0]=st_list[1]
            fielders.setdefault(i,[]).append((j[0],{'st1':st_list[0],'b1':st_list[1]}))
                
            pattern_st=None

In [ ]:
with open('fielder1.json', 'w') as fp:
    json.dump(fielders, fp, indent = 4)

### Mapped

In [ ]:
attr= pd.read_csv('Data/attributes.csv')

In [ ]:
attr.head()

In [ ]:
len(attr['Full Name'].unique())

In [ ]:
len(attr)

In [ ]:
attr['Full Name'].value_counts()

In [ ]:
attr.columns = ['FullName','Born','Bats' ,'Bowls','Code']

In [ ]:
attr['FullName']=attr['FullName'].str.lower()

In [ ]:
fielders={} # Match id : [(player id of player dismissed, c/st, b)]

In [ ]:
data_keys = list(data.keys())

In [ ]:
for i in data_keys:
    
    for j in data[i]:
        search_in=j[1].replace('\u2020','')
        pattern_catch = re.search('^c\s+',search_in)
        pattern_st = re.search('^st\s+',search_in)
        
        if pattern_catch:
            
            splitted = re.split('c\s+|\s+b\s+',search_in)
            catch_list = list(filter(None, splitted))
            if (len(catch_list)<2):
                catch_list.append(catch_list[0])
            if (catch_list[0]=='&'):
                catch_list[0]=catch_list[1]
            c_id=attr[attr['FullName'].str.contains(catch_list[0].lower())]['PlayerId']
            b_id=attr[attr['FullName'].str.contains(catch_list[1].lower())]['PlayerId']
            
            if (len(c_id)==1 and len(b_id)==1):
                fielders.setdefault(i,[]).append((j[0],{'c':int(c_id.iloc[0]),'b':int(b_id.iloc[0])}))
            else:
                if(len(c_id)>1):
                    id1=[pl for pl in c_id if pl in players[i]]
                    if(len(id1)==1):
                        catch_list[0]=id1
                   
                if(len(b_id)>1):
                    id2=[pl for pl in b_id if pl in players[i]]
                    if(len(id2)==1):
                        catch_list[1]=id2
                
                if(len(c_id)==1):
                    catch_list[0]=int(c_id.iloc[0])
                
                if(len(b_id)==1):
                    catch_list[1]=int(b_id.iloc[0])
                        
            fielders.setdefault(i,[]).append((j[0],{'c2':catch_list[0],'b2':catch_list[1]}))
            
            pattern_catch=None
            
        elif pattern_st:
            splitted = re.split('st\s+|\s+b\s+',search_in)
            st_list = list(filter(None, splitted))
            if (len(st_list)<2):
                st_list.append(st_list[0])
            if (st_list[0]=='&'):
                st_list[0]=st_list[1]
            st_id=attr[attr['FullName'].str.contains(st_list[0].lower())]['PlayerId']
            b1_id=attr[attr['FullName'].str.contains(st_list[1].lower())]['PlayerId']
            
            if (len(st_id)==1 and len(b1_id)==1):
                fielders.setdefault(i,[]).append((j[0],{'st':int(st_id.iloc[0]),'b':int(b1_id.iloc[0])}))
                
            else:
                if(len(st_id)>1):
                    id1=[pl for pl in st_id if pl in players[i]]
                    if(len(id1)==1):
                        st_list[0]=id1
                   
                if(len(b1_id)>1):
                    id2=[pl for pl in b1_id if pl in players[i]]
                    if(len(id2)==1):
                        st_list[1]=id2
                
                if(len(st_id)==1):
                    st_list[0]=int(st_id.iloc[0])
                
                if(len(b1_id)==1):
                    st_list[1]=int(b1_id.iloc[0])
                        
            fielders.setdefault(i,[]).append((j[0],{'st2':st_list[0],'b2':st_list[1]}))
                
            pattern_st=None

In [ ]:
with open('fielders.json', 'w') as fp:
    json.dump(fielders, fp,indent=4)